# RLDT

## Step 1: Import the necessary libraries:

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [3]:
num_IOT_device = 10

voltages_frequencies_IOT = [
    (1e6, 1.8),
    (2e6, 2.3),
    (4e6, 2.7),
    (8e6, 4.0),
    (16e6, 5.0),
    (32e6, 6.5),
]
num_MEC_devices = 5

voltages_frequencies_MEC = [
    (6 * 1e8, 0.8),
    (7.5 * 1e8, 0.825),
    (10 * 1e8, 1.0),
    (15 * 1e8, 1.2),
    (30 * 1e8, 2),
    (40 * 1e8, 3.1),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 10000

#### *IOT*

In [4]:
devices_data_IOT = []
for i in range(num_IOT_device):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            [
                voltages_frequencies_IOT[i]
                for i in np.random.choice(6, size=4, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "ISL": np.random.randint(10, 21),
        "capacitance": [np.random.uniform(2, 3) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([700, 800, 900]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices

NameError: name 'num_IOT_devices' is not defined

#### *MEC*

In [ ]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [[
            voltages_frequencies_MEC[i]
            for i in np.random.choice(6, size=4, replace=False)
        ]for core in range(cpu_cores)],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([9, 9, 10]) * 1e-5 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices

#### *CLOUD*

In [ ]:
cloud_configurations = [(1,13.85), (2,3.9e9,24.28)]
device_info = [
    {
        "id": 0,
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [[2.8e9, 3.9e9]],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "accetableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)
cloud=cloud.set_index("id")
cloud

####        ALL THE DEVICES

In [ ]:
devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
devices

### Step 2.2: Application

#### *helper function : generate_random_dag*

In [ ]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [ ]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)
    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "job": run,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
            "dataEntrySize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "returnDataSize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)
tasks

## Step 3: Preprocessing

### Step 3.1: Clustering

In [ ]:
devices

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


def calculate_mean_voltage_frequency(input):
    output = 0
    for x in input:
        if len(x) == 2:
            output += (x[0] + x[1]) / 2
        else:
            output += sum([vf[0] / vf[1] for vf in x]) / len(x)
    return output / len(input)


selected_features = [
    "number_of_cpu_cores",
    "voltages_frequencies",
    "capacitance",
    "ISL",
    "batteryLevel",
    "errorRate",
    "accetableTasks",
]

X = devices[selected_features].copy()
X["mean_voltage_frequency"] = X["voltages_frequencies"].apply(
    calculate_mean_voltage_frequency
)
X["mean_capacitance"] = X["capacitance"].apply(lambda x: sum(x) / len(x))
X["meanBattery"] = X["batteryLevel"] * (1 - X["ISL"])
X["doesAcceptT1"] = X["accetableTasks"].apply(lambda x: 1 in x)
X["doesAcceptT2"] = X["accetableTasks"].apply(lambda x: 2 in x)
X["doesAcceptT3"] = X["accetableTasks"].apply(lambda x: 3 in x)
X["doesAcceptT4"] = X["accetableTasks"].apply(lambda x: 4 in x)

X.drop(
    ["voltages_frequencies", "capacitance", "batteryLevel", "ISL", "accetableTasks"],
    axis=1,
    inplace=True,
)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_clusters = 8

kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
kmeans.fit(X_scaled)
cluster_labels = kmeans.labels_

devices["cluster"] = cluster_labels
devices

## Step 4 : DDT

### Step 4.1:  Initializing The tree

In [ ]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            self.weights = nn.Parameter(torch.zeros(num_input))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist.softmax(dim=0)
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if val>=0.5:
            if a < 0.10:
                return val * self.left(x)
            return val * self.right(x)
        else:
            if a < 0.10:
                return val * self.right(x)
            return val * self.left(x)

## Step 5: RL

In [ ]:
def getStatusOfDependcy(dependency):
    result = 0
    for dep in dependency:
        result += tasks_copy.loc[dep, "status"]
    return result


tasks_copy = tasks.copy()
tasks_copy["status"] = tasks_copy["status"].map(
    {"WAIT": 0, "READY": 1, "QUEUED": 2, "SCEDULED": 3}
)
tasks_copy["dependency"] = tasks_copy["dependency"].apply(getStatusOfDependcy)

taskList = tasks_copy.index.tolist()
devices_copy = devices.copy()

In [ ]:
def calc_execution_time(device, task, core,dvfs):
    if device['number_of_cpu_cores'] ==1:
        return task["computationalLoad"] / device["voltages_frequencies"][0][0]
    return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device,task,core,dvfs):
    if device["number_of_cpu_cores"] == 1:
       return device["capacitance"][0] * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * calc_execution_time(device, task, core, dvfs)
    )

In [ ]:
tasks_copy

In [ ]:
taskList = tasks_copy.index.tolist()

class Environment:
    def __init__(self):
        self.feature_size = 9
        self.num_actions = 8
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions, depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        taskList.pop(0)

        selected_cluster = devices_copy[devices_copy["cluster"] == action]
        for row, device in selected_cluster.iterrows():
            checkAvailableCoree = (sum(device["occupied_cores"]) != device["number_of_cpu_cores"])
            checkIfSutible = state['kind'] in device['accetableTasks'] 
            checkBattery = device['batteryLevel'] > device['ISL']*2e8
            if (
                checkAvailableCoree
                and checkIfSutible
                and checkBattery
                and np.random.rand() > device['errorRate']
            ):  
                for coreIndex in range(len(device["occupied_cores"])):
                    if device["occupied_cores"][coreIndex] ==0:
                        e = calc_power_consumption(device, state,coreIndex, 0)
                        t = calc_execution_time(device, state,coreIndex, 0)
                        return (tasks_copy.loc[taskList[0]], -1 * t + -1 * e)
        return (tasks_copy.loc[taskList[0]],100)

    def train(self, num_epoch, num_episodes):
        for i in range(num_epoch):
            total_loss = 0
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)

                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)

                # Sample an action based on the output probabilities
                action_index = torch.multinomial(torch.softmax(output, dim=0), 1).item()
                next_state, reward = self.execute_action(state, action_index)

                # Calculate the loss as the negative log probability of the chosen action multiplied by the reward
                loss = (
                    output[action_index] * reward
                ) 
                print(output,action_index,reward,loss)
                total_loss += loss
            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_loss.backward()
            self.optimizer.step()
            print(f"Epoch {i+1}, Average Loss: {avg_loss}")
            # for name, param in env.agent.named_parameters():
            # print(name, param.grad)


env = Environment()
tree = env.agent
env.train(2, 10)
print('///////////////////')
for name, param in env.agent.named_parameters():
    if "prob_dist" not in name:
        print(name,param)